In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Input, Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPool2D
from matplotlib import pyplot as plt

In [ ]:
# Download the Pylearn2/Keras implementation of LocalResponseNormalization.
!curl -sO https://raw.githubusercontent.com/mbrukman/reimplementing-ml-papers/main/third_party/pylearn2/local_response_normalization.py

from local_response_normalization import LRN2D

In [ ]:
# Load the CIFAR-10 dataset.
(x_train_raw, y_train_raw), (x_test_raw, y_test_raw) = keras.datasets.cifar10.load_data();

170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
num_classes = 10

# Scale train and test inputs by converting them from range of [0, 255] to [0.0, 1.0]
x_train = x_train_raw.astype('float32') / 255.0
x_test = x_test_raw.astype('float32') / 255.0

# Convert the output to categorical one-hot encoding to match the output of our
# network.
y_train = keras.utils.to_categorical(y_train_raw, num_classes)
y_test = keras.utils.to_categorical(y_test_raw, num_classes)

In [ ]:
# Define the model architecture, this comes from the the Google Code project
# mentioned in the paper — https://code.google.com/p/cuda-convnet/ —
# specifically, the file `examples-layers/layers-conv-local-11pct.cfg`.
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu', name='Conv1'),
    MaxPool2D(pool_size=3, strides=2, padding='valid', name='MaxPool1'),
    LRN2D(name='LRN1'),
    Conv2D(filters=64, kernel_size=(5, 5), padding='same', activation='relu', name='Conv2'),
    LRN2D(name='LRN2'),
    MaxPool2D(pool_size=3, strides=2, padding='valid', name='MaxPool2'),
    Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', name='Local3'),
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', name='Local4'),
    Flatten(),
    Dense(10, activation='softmax', name="FC10"),
], name="CIFAR-10-Pylearn2-Keras-LRN2D")

model.summary()

Model: "CIFAR-10-Pylearn2-Keras-LRN2D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 32, 32, 64)        4864      
                                                                 
 MaxPool1 (MaxPooling2D)     (None, 15, 15, 64)        0         
                                                                 
 LRN1 (LRN2D)                (None, 15, 15, 64)        0         
                                                                 
 Conv2 (Conv2D)              (None, 15, 15, 64)        102464    
                                                                 
 LRN2 (LRN2D)                (None, 15, 15, 64)        0         
                                                                 
 MaxPool2 (MaxPooling2D)     (None, 7, 7, 64)          0         
                                                                 
 Local3 (Conv2D)             (None, 7

In [ ]:
# Compile the model with optimizer and loss function.
opt = keras.optimizers.Adam(learning_rate=0.001)
loss_fn = keras.losses.CategoricalCrossentropy()
model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=20)

Epoch 1/20
1563/1563 [==============================] - 334s 213ms/step - loss: 1.5085 - accuracy: 0.4538
Epoch 2/20
1563/1563 [==============================] - 333s 213ms/step - loss: 1.1214 - accuracy: 0.6022
Epoch 3/20
1563/1563 [==============================] - 335s 214ms/step - loss: 0.9469 - accuracy: 0.6651
Epoch 4/20
1563/1563 [==============================] - 345s 221ms/step - loss: 0.8448 - accuracy: 0.7037
Epoch 5/20
1563/1563 [==============================] - 333s 213ms/step - loss: 0.7661 - accuracy: 0.7308
Epoch 6/20
1563/1563 [==============================] - 333s 213ms/step - loss: 0.7095 - accuracy: 0.7527
Epoch 7/20
1563/1563 [==============================] - 333s 213ms/step - loss: 0.6607 - accuracy: 0.7681
Epoch 8/20
1563/1563 [==============================] - 332s 212ms/step - loss: 0.6190 - accuracy: 0.7817
Epoch 9/20
1563/1563 [==============================] - 333s 213ms/step - loss: 0.5824 - accuracy: 0.7961
Epoch 10/20
1563/1563 [=======================

In [ ]:
# Evaluate the model
model.evaluate(x_test, y_test)

313/313 [==============================] - 16s 51ms/step - loss: 1.3028 - accuracy: 0.6922


[1.3027948141098022, 0.6922000050544739]